In [1]:
import sys
from pathlib import Path

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper
import tasks.preprocessing

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df["random"] = tasks.preprocessing.get_rand_col(df, "annot_sex")
df = df.reset_index()


In [3]:
sdb_columns = [
    "annot_age",
    "annot_sex",
    "annot_demographic_group",
    "annot_sexual_orientation",
    "annot_current_employment",
    "annot_education_level",
    "annot_politics"
]
res = tasks.run_helper.run_all_results(
    df=df,
    sdb_columns=sdb_columns,
    value_col="toxicity",
    comment_key_col="comment_key",
)
res

Evaluating SDB dimensions:   0%|          | 0/7 [00:00<?, ?it/s]

0         1
sdb_column                                                          
annot_age                (14.927, 33.25]          0.250640  0.816667
                         (33.25, 51.5]            0.361527  0.816667
                         (51.5, 69.75]           -0.990521  0.816667
                         (69.75, 88.0]           -0.170735  0.816667
annot_current_employment blue-collar             -0.160093  0.800000
                         unemployed               0.654067  0.800000
                         white-collar            -0.458914  0.800000
annot_demographic_group  asian                    1.434277  0.816667
                         black                    0.541693  0.816667
                         other                   -2.479864  0.816667
                         white                    1.043412  0.816667
annot_education_level    high-school              0.936387  0.800000
                         none                    -1.440898  0.800000
                         university               0.287327  0.800000
annot_politics           apolitical              -2.519218  0.833333
                         left-wing liberal        1.050793  0.833333
                         right-wing conservative  0.620836  0.833333
annot_sex                female                  -0.188558  0.800000
                         male                    -0.522449  0.800000
                         non-binary               3.737803  0.800000
annot_sexual_orientation bisexual                 3.595870  0.833333
                         homosexual              -0.265278  0.833333
                         other                   -2.871549  0.833333
                         straight                 0.343202  0.833333

In [4]:
tasks.run_helper.run_result(
    df,
    sdb_column="random",
    value_col="toxicity",
    comment_key_col="comment_key",
)

,0,1
4,-0.017322,0.816667
2,0.511258,0.816667
1,-1.298453,0.816667
3,-0.287349,0.816667
